In [1]:
import os
import tensorflow as tf
import magenta.music as mm
from magenta.models.music_vae import TrainedModel, configs
from magenta.music import DEFAULT_STEPS_PER_BAR
from magenta.music.protobuf.music_pb2 import NoteSequence
from six.moves import urllib
from typing import List
from note_sequence_utils import save_midi, save_plot

print(tf.test.gpu_device_name())

/device:GPU:0


In [2]:
# Download Magenta checkpoint to target directory
def download_checkpoint(model, checkpoint, target_dir):
    
    tf.io.gfile.makedirs(target_dir)
    checkpoint_target = os.path.join(target_dir, checkpoint)
    
    if not os.path.exists(checkpoint_target):
        
        response = urllib.request.urlopen("https://storage.googleapis.com/magentadata/models/{}/checkpoints/{}".format(model, checkpoint))
        data = response.read()
        file = open(checkpoint_target, 'wb')
        file.write(data)
        file.close()

In [3]:
# Instantiate the model
def get_model(model):
    
    checkpoint = model + ".tar"
    download_checkpoint("music_vae", checkpoint, "checkpoints")
    
    return TrainedModel(configs.CONFIG_MAP[model.split(".")[0] if "." in model else model],
                        batch_size = 8, checkpoint_dir_or_path = os.path.join("checkpoints", checkpoint))

In [4]:
# Sample method
def sample(model_name, steps_per_sample):
    
    model = get_model(model_name)
    sample_seq = model.sample(n = 2, length = steps_per_sample, temperature = 1.1)
    
    save_midi(sample_seq, "sample", model_name)
    save_plot(sample_seq, "sample", model_name)
    
    return sample_seq

In [5]:
# Interpolate method -> takes two sequeneces and interpolates between them
def interpolate(model_name, sample_seq : List[NoteSequence], steps_per_sample, n_output, n_bars):
    
    if len(sample_seq) != 2:
        raise Exception('Expected 2 sequences, actual sequences : {}'.format(len(sample_seq)))
    
    if not sample_seq[0].notes or not sample_seq[1].notes:
        raise Exception('None. \nSequence 1 has length : {} \nSequence 2 has length : {}'.format(
        len(sample_seq[0].notes), len(sample_seq[1].notes)))
        
    model = get_model(model_name)
    
    interpolate_sequences = model.interpolate(start_sequence = sample_seq[0], 
                                        end_sequence = sample_seq[1],
                                        num_steps = n_output,
                                        length = steps_per_sample)
    save_midi(interpolate_sequences, "interpolate", model_name)
    save_plot(interpolate_sequences, "interpolate", model_name)
    
    interpolate_seq = mm.sequences_lib.concatenate_sequences(interpolate_sequences, [4] * n_output)
    save_midi(interpolate_seq, "merge", model_name)
    save_plot(interpolate_seq, "merge", model_name, plot_max_length_bar = n_bars, bar_fill_alphas = [0.50, 0.50, 0.05, 0.05])
    
    return interpolate_seq

In [6]:
# Groove method -> adds groove to interpolated sequence by splitting in chunks
def groove(model_name, interpolate_seq, steps_per_sample, n_output, n_bars):
    
    model = get_model(model_name)
    
    split_interpolated_seq = mm.sequences_lib.split_note_sequence(interpolate_seq, 4)
    
    if len(split_interpolated_seq) != n_output:
        raise Exception('Wrong interpolation size. Expected is 10, actual is {}'.format(split_interpolated_seq))
            
    encoding, mu, sigma = model.encode(note_sequences = split_interpolated_seq)
    
    groove_sequences = model.decode(z = encoding, length = steps_per_sample)
    groove_seq = mm.sequences_lib.concatenate_sequences(groove_sequences, [4] * n_output)
    
    save_midi(groove_seq, "groove", model_name)
    save_plot(groove_seq, "groove", model_name, plot_max_length_bar = n_bars, 
              show_velocity = True, bar_fill_alphas = [0.50, 0.50, 0.05, 0.05])
    
    return groove_seq

In [7]:
n_output = 6
n_bar_per_sample = 2
n_steps_per_sample = n_bar_per_sample * DEFAULT_STEPS_PER_BAR
n_bars = n_output * n_bar_per_sample

In [8]:
generated_sample_sequences = sample("cat-drums_2bar_small.lokl", n_steps_per_sample)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 8, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

Instructions for updating

In [9]:
generate_interpolate_seq = interpolate("cat-drums_2bar_small.hikl",
                                       generated_sample_sequences, 
                                       n_steps_per_sample, n_output, n_bars)

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 256, 'free_bits': 48, 'max_beta': 0.2, 'beta_rate': 0.0, 'batch_size': 8, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [256, 256], 'enc_rnn_size': [512], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [512]

INFO:tensorflow:
Decoder Cells:
  units: [256, 256]

INFO:tensorflow:Unbundling checkpoint.
INFO:tensorflow:Restoring parameters from C:\Users\siddh\AppData\Local\Temp\tmppt3sgipj\cat-drums_2bar_small.hikl.ckpt
Generated midi : C:\Users\siddh\Documents\GitHub\magenta_project\output\interpolate\

In [ ]:
generate_groove_sequece = groove("groovae_2bar_humanize",
                                 generate_interpolate_seq,
                                 n_steps_per_sample, n_output, n_bars)